In [1]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [2]:
chess_oh = pd.read_csv('chess_oh.csv')
chess_oh.head()

,wking_file_a,wking_file_b,wking_file_c,wking_file_d,wking_rank_1,wking_rank_2,wking_rank_3,wking_rank_4,wrook_file_a,wrook_file_b,...,bking_file_h,bking_rank_1,bking_rank_2,bking_rank_3,bking_rank_4,bking_rank_5,bking_rank_6,bking_rank_7,bking_rank_8,Y
0,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


## Create 5 trials, each with train and test set

In [4]:
from sklearn.model_selection import train_test_split

XY_train1, XY_test1 = train_test_split(chess_oh, train_size=5000, shuffle=True)
XY_train2, XY_test2 = train_test_split(chess_oh, train_size=5000, shuffle=True)
XY_train3, XY_test3 = train_test_split(chess_oh, train_size=5000, shuffle=True)
XY_train4, XY_test4 = train_test_split(chess_oh, train_size=5000, shuffle=True)
XY_train5, XY_test5 = train_test_split(chess_oh, train_size=5000, shuffle=True)

In [5]:
print(XY_train1.shape)
print(XY_test1.shape)
XY_train1.head()

(5000, 41)
(23056, 41)


,wking_file_a,wking_file_b,wking_file_c,wking_file_d,wking_rank_1,wking_rank_2,wking_rank_3,wking_rank_4,wrook_file_a,wrook_file_b,...,bking_file_h,bking_rank_1,bking_rank_2,bking_rank_3,bking_rank_4,bking_rank_5,bking_rank_6,bking_rank_7,bking_rank_8,Y
22252,0,0,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
272,0,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
12616,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1650,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
11974,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Perform GridSearchCV on classifier

In [6]:
%%time
#import warnings
# there are a lot of convergence warnings for some params, however be careful with this!!
# sometimes you need to see those wanrings, and now we've screwed tha tup for the whole notebook from here on!!
#warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV

# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', KNeighborsClassifier())])

# Create search space of candidate learning algorithms and their hyperparameters
grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

trialnum = 0
accuracy_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
roc_sum = 0 # sum of top roc score to later calculate the average of all 5 trials
f1_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
accuracy_scores = []
roc_scores = []
f1_scores = []
all_accuracy_models = []
all_roc_models = []
all_f1_models = []

# for every trial
for trial in [XY_train1, XY_train2, XY_train3, XY_train4, XY_train5]:
 
    trialnum = trialnum + 1
    X_l = trial.drop(['Y'],1)
    y_l = trial['Y']

    clf = GridSearchCV(KNeighborsClassifier(), param_grid=grid_params, cv=StratifiedKFold(n_splits=5), 
                      scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False, 
                       n_jobs=-1, verbose=-1)

    # Fit grid search
    best_model = clf.fit(X_l, y_l)
    
    print("------------------------------------------------------------------------------------")
    print("RESULTS FOR TRIAL:")
    print(trialnum)
    print("------------------------------------------------------------------------------------")
    
    # the detailed results of the whole model selection search...
#     print(best_model.cv_results_)

    print("---------------BEST MODEL FOR ACCURACY: ----------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    print("---WITH ACCURACY: ---")
    current_accuracy = best_model.cv_results_['mean_test_accuracy'][ np.argmax(best_model.cv_results_['mean_test_accuracy']) ]
    print(current_accuracy)
    accuracy_sum = accuracy_sum + current_accuracy
    accuracy_scores.append(current_accuracy)
    all_accuracy_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    
    print("---------------BEST MODEL FOR ROC: ---------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )
    print("---WITH ROC: ---")
    current_roc = best_model.cv_results_['mean_test_roc_auc_ovr'][ np.argmax(best_model.cv_results_['mean_test_roc_auc_ovr']) ]
    print(current_roc)
    roc_sum = roc_sum + current_roc
    roc_scores.append(current_roc)
    all_roc_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )

    
    print("---------------BEST MODEL FOR F1: ----------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )
    print("---WITH F1: ---")    
    current_f1 = best_model.cv_results_['mean_test_f1_micro'][ np.argmax(best_model.cv_results_['mean_test_f1_micro']) ]
    print(current_f1)
    f1_sum = f1_sum + current_f1
    f1_scores.append(current_f1)
    all_f1_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )


------------------------------------------------------------------------------------
RESULTS FOR TRIAL:
1
------------------------------------------------------------------------------------
---------------BEST MODEL FOR ACCURACY: ----------
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
---WITH ACCURACY: ---
0.8522000000000001
---------------BEST MODEL FOR ROC: ---------------
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
---WITH ROC: ---
0.9278945365199214
---------------BEST MODEL FOR F1: ----------------
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
---WITH F1: ---
0.8522000000000001
------------------------------------------------------------------------------------
RESULTS FOR TRIAL:
2
------------------------------------------------------------------------------------
---------------BEST MODEL FOR ACCURACY: ----------
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
---WITH ACCURACY: ---
0.85039999999999

### Get Train metrics

In [7]:
print("===== ACCURACY SCORES: =====")
print(accuracy_scores)
print("===== ROC SCORES: =====")
print(roc_scores)
print("===== F1 SCORES: =====")
print(f1_scores)

===== ACCURACY SCORES: =====
[0.8522000000000001, 0.8503999999999999, 0.842, 0.8486, 0.85]
===== ROC SCORES: =====
[0.9278945365199214, 0.9295282440075336, 0.9224028208840467, 0.929108595836999, 0.9280155579543168]
===== F1 SCORES: =====
[0.8522000000000001, 0.8503999999999999, 0.842, 0.8486, 0.85]


In [8]:
print("================ BEST ACCURACY MODEL IN TRAINING: ==================")
print(all_accuracy_models[ np.argmax(accuracy_scores) ])
print("WITH ACCURACY:")
print(max(accuracy_scores))
print("================ BEST ROC MODEL IN TRAINING: ==================")
print(all_roc_models[ np.argmax(roc_scores) ])
print("WITH ROC SCORE:")
print(max(roc_scores))
print("================ BEST F1 MODEL IN TRAINING: ==================")
print(all_f1_models[ np.argmax(f1_scores) ])
print("WITH F1 SCORE:")
print(max(f1_scores))

================ BEST ACCURACY MODEL IN TRAINING: ==================
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
WITH ACCURACY:
0.8522000000000001
================ BEST ROC MODEL IN TRAINING: ==================
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
WITH ROC SCORE:
0.9295282440075336
================ BEST F1 MODEL IN TRAINING: ==================
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
WITH F1 SCORE:
0.8522000000000001


In [9]:
print("================ AVERAGE ACCURACY ON TRAIN SET: ==================")
print(accuracy_sum / 5)
print("================ AVERAGE ROC SCORE ON TRAIN SET: ==================")
print(roc_sum / 5)
print("================ AVERAGE F1 SCORE ON TRAIN SET: ==================")
print(f1_sum / 5)

================ AVERAGE ACCURACY ON TRAIN SET: ==================
0.84864
================ AVERAGE ROC SCORE ON TRAIN SET: ==================
0.9273899510405634
================ AVERAGE F1 SCORE ON TRAIN SET: ==================
0.84864


### Get Test metrics

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# test performance of model rated as best for accuracy and f1 score on training set
performance_AccModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])
performance_AccModel

# test performance on trial 1 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance').fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_AccModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_AccModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance').fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_AccModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_AccModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance').fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_AccModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_AccModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance').fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_AccModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_AccModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance').fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_AccModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_AccModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_AccModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.849627,0.845723,0.853314,0.843642,0.848976
roc,0.849528,0.845662,0.853232,0.843673,0.848851
f1,0.846762,0.841651,0.849942,0.838406,0.845601


In [11]:
# test performance of model rated as best for roc on training set
performance_ROCModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])

# test performance on trial 1 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance').fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance').fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance').fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance').fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance').fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_ROCModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.84876,0.844422,0.853574,0.846634,0.849714
roc,0.848645,0.84435,0.853491,0.846668,0.849566
f1,0.845605,0.83963,0.850142,0.84115,0.845938


## Get final KNN results

In [12]:
print("================ BEST ACCURACY MODEL IN KNN: ==================")
print(all_accuracy_models[ np.argmax(np.array(performance_AccModel.iloc[0])) ])
print("WITH ACCURACY:")
print((np.array(performance_AccModel.iloc[0])).mean())
print("================ BEST ROC MODEL IN KNN: ==================")
print(all_roc_models[ np.argmax(np.array(performance_ROCModel.iloc[1])) ])
print("WITH ROC SCORE:")
print((performance_ROCModel.iloc[1]).mean())
print("================ BEST F1 MODEL IN KNN: ==================")
print(all_f1_models[ np.argmax(np.array(performance_AccModel.iloc[2])) ])
print("WITH F1 SCORE:")
print((np.array(performance_AccModel.iloc[2])).mean())

================ BEST ACCURACY MODEL IN KNN: ==================
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
WITH ACCURACY:
0.8482564191533658
================ BEST ROC MODEL IN KNN: ==================
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
WITH ROC SCORE:
0.8485439141942365
================ BEST F1 MODEL IN KNN: ==================
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
WITH F1 SCORE:
0.8444725533216999
